<a href="https://colab.research.google.com/github/Mohona77/Audio-Classification-SER-/blob/main/Imocap_Model.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [1]:
pip install transformers datasets evaluate

   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 480.6/480.6 kB 8.1 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 84.0/84.0 kB 5.7 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 116.3/116.3 kB 7.4 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 179.3/179.3 kB 11.4 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 134.8/134.8 kB 8.9 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 194.1/194.1 kB 14.7 MB/s eta 0:00:00
  Attempting uninstall: fsspec
    Found existing installation: fsspec 2024.10.0
    Uninstalling fsspec-2024.10.0:
      Successfully uninstalled fsspec-2024.10.0
ERROR: pip's dependency resolver does not currently take into account all the packages that are installed. This behaviour is the source of the following dependency conflicts.
gcsfs 2024.10.0 requires fsspec==2024.10.0, but you have fsspec 2024.9.0 which is incompatible.


In [2]:
from huggingface_hub import notebook_login

notebook_login()

# **Loading Base wav2vec2 model**

In [3]:
from transformers import AutoFeatureExtractor

feature_extractor = AutoFeatureExtractor.from_pretrained("facebook/wav2vec2-base")

/usr/local/lib/python3.10/dist-packages/huggingface_hub/utils/_token.py:89: UserWarning: 
The secret `HF_TOKEN` does not exist in your Colab secrets.
To authenticate with the Hugging Face Hub, create a token in your settings tab (https://huggingface.co/settings/tokens), set it as secret in your Google Colab and restart your session.
You will be able to reuse this secret in all of your notebooks.
Please note that authentication is recommended but still optional to access public models or datasets.
  warnings.warn(


preprocessor_config.json:   0%|          | 0.00/159 [00:00<?, ?B/s]

config.json:   0%|          | 0.00/1.84k [00:00<?, ?B/s]

/usr/local/lib/python3.10/dist-packages/transformers/configuration_utils.py:364: UserWarning: Passing `gradient_checkpointing` to a config initialization is deprecated and will be removed in v5 Transformers. Using `model.gradient_checkpointing_enable()` instead, or if you are using the `Trainer` API, pass `gradient_checkpointing=True` in your `TrainingArguments`.
  warnings.warn(


In [4]:
%%capture
!pip install gdown

In [5]:
import os
import zipfile
from google.colab import files
import time
import json
import librosa
from datasets import Dataset, DatasetDict, Audio
import glob
import pandas as pd
import warnings
warnings.filterwarnings("ignore")

In [6]:
!gdown --id 16eCv6Sa4hklMoEk3ahp4lc1V561Elrml
time.sleep(5)
!unzip /content/IEMOCAP_splits.zip

/usr/local/lib/python3.10/dist-packages/gdown/__main__.py:140: FutureWarning: Option `--id` was deprecated in version 4.3.1 and will be removed in 5.0. You don't need to pass it anymore to use a file ID.
  warnings.warn(
Downloading...
From: https://drive.google.com/uc?id=16eCv6Sa4hklMoEk3ahp4lc1V561Elrml
To: /content/IEMOCAP_splits.zip
100% 817k/817k [00:00<00:00, 84.3MB/s]
Archive:  /content/IEMOCAP_splits.zip
   creating: iemocap/
   creating: iemocap/session5/
  inflating: iemocap/session5/test.json  
  inflating: iemocap/session5/train.json  
   creating: iemocap/session2/
  inflating: iemocap/session2/test.json  
  inflating: iemocap/session2/train.json  
   creating: iemocap/session3/
  inflating: iemocap/session3/test.json  
  inflating: iemocap/session3/train.json  
   creating: iemocap/session4/
  inflating: iemocap/session4/test.json  
  inflating: iemocap/session4/train.json  
   creating: iemocap/session1/
  inflating: iemocap/session1/test.json  
  inflating: iemocap/sess

In [7]:
import json

# Paths to train and test JSON files
# train_json_path = os.path.join(extract_path, 'train.json')
# test_json_path = os.path.join(extract_path, 'test.json')

# Load JSON files
with open("/content/iemocap/session1/train.json",'r') as f:
    train_data = json.load(f)

with open("/content/iemocap/session1/test.json",'r') as f:
    test_data = json.load(f)

In [8]:
train_data

{'Ses02F_impro03_F000': {'wav': '/m/teamwork/t40511_asr/c/IEMOCAP_full_release/Session2/sentences/wav/Ses02F_impro03/Ses02F_impro03_F000.wav',
  'length': 4.0475736961451245,
  'emo': 'hap'},
 'Ses02F_impro03_F001': {'wav': '/m/teamwork/t40511_asr/c/IEMOCAP_full_release/Session2/sentences/wav/Ses02F_impro03/Ses02F_impro03_F001.wav',
  'length': 2.119954648526077,
  'emo': 'hap'},
 'Ses02F_impro03_F004': {'wav': '/m/teamwork/t40511_asr/c/IEMOCAP_full_release/Session2/sentences/wav/Ses02F_impro03/Ses02F_impro03_F004.wav',
  'length': 2.0,
  'emo': 'hap'},
 'Ses02F_impro03_F009': {'wav': '/m/teamwork/t40511_asr/c/IEMOCAP_full_release/Session2/sentences/wav/Ses02F_impro03/Ses02F_impro03_F009.wav',
  'length': 12.08702947845805,
  'emo': 'hap'},
 'Ses02F_impro03_F011': {'wav': '/m/teamwork/t40511_asr/c/IEMOCAP_full_release/Session2/sentences/wav/Ses02F_impro03/Ses02F_impro03_F011.wav',
  'length': 4.207210884353741,
  'emo': 'hap'},
 'Ses02F_impro03_F012': {'wav': '/m/teamwork/t40511_asr/c/

In [9]:
 test_data

{'Ses01F_script02_2_F000': {'wav': '/m/teamwork/t40511_asr/c/IEMOCAP_full_release/Session1/sentences/wav/Ses01F_script02_2/Ses01F_script02_2_F000.wav',
  'length': 2.478140589569161,
  'emo': 'neu'},
 'Ses01F_script02_2_F001': {'wav': '/m/teamwork/t40511_asr/c/IEMOCAP_full_release/Session1/sentences/wav/Ses01F_script02_2/Ses01F_script02_2_F001.wav',
  'length': 1.872517006802721,
  'emo': 'neu'},
 'Ses01F_script02_2_F006': {'wav': '/m/teamwork/t40511_asr/c/IEMOCAP_full_release/Session1/sentences/wav/Ses01F_script02_2/Ses01F_script02_2_F006.wav',
  'length': 6.289977324263038,
  'emo': 'ang'},
 'Ses01F_script02_2_F008': {'wav': '/m/teamwork/t40511_asr/c/IEMOCAP_full_release/Session1/sentences/wav/Ses01F_script02_2/Ses01F_script02_2_F008.wav',
  'length': 5.994512471655328,
  'emo': 'ang'},
 'Ses01F_script02_2_F009': {'wav': '/m/teamwork/t40511_asr/c/IEMOCAP_full_release/Session1/sentences/wav/Ses01F_script02_2/Ses01F_script02_2_F009.wav',
  'length': 4.2,
  'emo': 'ang'},
 'Ses01F_scrip

In [10]:
train_samples_list = [
    {"id": sample_id, **sample_data} for sample_id, sample_data in train_data.items()
]
train_dataset = Dataset.from_list(train_samples_list)

In [11]:
test_samples_list = [
    {"id": sample_id, **sample_data} for sample_id, sample_data in test_data.items()
]
test_dataset = Dataset.from_list(test_samples_list)

In [12]:
dataset = DatasetDict({
    'train' : train_dataset,
    'test' : test_dataset,
    'dev' : test_dataset
})
dataset

DatasetDict({
    train: Dataset({
        features: ['id', 'wav', 'length', 'emo'],
        num_rows: 4446
    })
    test: Dataset({
        features: ['id', 'wav', 'length', 'emo'],
        num_rows: 1085
    })
    dev: Dataset({
        features: ['id', 'wav', 'length', 'emo'],
        num_rows: 1085
    })
})

In [22]:
import os
import librosa

def add_base_path(example):
    # Assuming the original paths are relative to /content/iemocap
    if not example['wav'].startswith('/content/iemocap'):
        example['wav'] = os.path.join('/content/iemocap', example['wav'])
    print(f"Updated path: {example['wav']}")  # Debugging line
    return example

def load_audio(example):
    try:
        audio_array, sample_rate = librosa.load(example['wav'], sr=16000)
        example["audio_array"] = audio_array
        example["sample_rate"] = sample_rate
        return example
    except FileNotFoundError:
        print(f"File not found: {example['wav']}")
        return None  # Skip this sample
    except Exception as e:
        print(f"Error loading {example['wav']}: {e}")
        return None  # Skip this sample

# Assuming dataset is already defined and loaded
dataset = dataset.map(add_base_path, batched=False)
dataset = dataset.map(load_audio, batched=False)
# Remove samples that failed to load
dataset = dataset.filter(lambda example: example is not None)


Map:   0%|          | 0/4446 [00:00<?, ? examples/s]

Updated path: /content/iemocap/session1/m/teamwork/t40511_asr/c/IEMOCAP_full_release/Session2/sentences/wav/Ses02F_impro03/Ses02F_impro03_F000.wav
Updated path: /content/iemocap/session1/m/teamwork/t40511_asr/c/IEMOCAP_full_release/Session2/sentences/wav/Ses02F_impro03/Ses02F_impro03_F001.wav
Updated path: /content/iemocap/session1/m/teamwork/t40511_asr/c/IEMOCAP_full_release/Session2/sentences/wav/Ses02F_impro03/Ses02F_impro03_F004.wav
Updated path: /content/iemocap/session1/m/teamwork/t40511_asr/c/IEMOCAP_full_release/Session2/sentences/wav/Ses02F_impro03/Ses02F_impro03_F009.wav
Updated path: /content/iemocap/session1/m/teamwork/t40511_asr/c/IEMOCAP_full_release/Session2/sentences/wav/Ses02F_impro03/Ses02F_impro03_F011.wav
Updated path: /content/iemocap/session1/m/teamwork/t40511_asr/c/IEMOCAP_full_release/Session2/sentences/wav/Ses02F_impro03/Ses02F_impro03_F012.wav
Updated path: /content/iemocap/session1/m/teamwork/t40511_asr/c/IEMOCAP_full_release/Session2/sentences/wav/Ses02F_imp

Map:   0%|          | 0/1085 [00:00<?, ? examples/s]

Updated path: /content/iemocap/session1/m/teamwork/t40511_asr/c/IEMOCAP_full_release/Session1/sentences/wav/Ses01F_script02_2/Ses01F_script02_2_F000.wav
Updated path: /content/iemocap/session1/m/teamwork/t40511_asr/c/IEMOCAP_full_release/Session1/sentences/wav/Ses01F_script02_2/Ses01F_script02_2_F001.wav
Updated path: /content/iemocap/session1/m/teamwork/t40511_asr/c/IEMOCAP_full_release/Session1/sentences/wav/Ses01F_script02_2/Ses01F_script02_2_F006.wav
Updated path: /content/iemocap/session1/m/teamwork/t40511_asr/c/IEMOCAP_full_release/Session1/sentences/wav/Ses01F_script02_2/Ses01F_script02_2_F008.wav
Updated path: /content/iemocap/session1/m/teamwork/t40511_asr/c/IEMOCAP_full_release/Session1/sentences/wav/Ses01F_script02_2/Ses01F_script02_2_F009.wav
Updated path: /content/iemocap/session1/m/teamwork/t40511_asr/c/IEMOCAP_full_release/Session1/sentences/wav/Ses01F_script02_2/Ses01F_script02_2_F010.wav
Updated path: /content/iemocap/session1/m/teamwork/t40511_asr/c/IEMOCAP_full_relea

Map:   0%|          | 0/1085 [00:00<?, ? examples/s]

...

Map:   0%|          | 0/1085 [00:00<?, ? examples/s]

File not found: /content/iemocap/session1/m/teamwork/t40511_asr/c/IEMOCAP_full_release/Session1/sentences/wav/Ses01F_script02_2/Ses01F_script02_2_F000.wav
File not found: /content/iemocap/session1/m/teamwork/t40511_asr/c/IEMOCAP_full_release/Session1/sentences/wav/Ses01F_script02_2/Ses01F_script02_2_F001.wav
File not found: /content/iemocap/session1/m/teamwork/t40511_asr/c/IEMOCAP_full_release/Session1/sentences/wav/Ses01F_script02_2/Ses01F_script02_2_F006.wav
File not found: /content/iemocap/session1/m/teamwork/t40511_asr/c/IEMOCAP_full_release/Session1/sentences/wav/Ses01F_script02_2/Ses01F_script02_2_F008.wav
File not found: /content/iemocap/session1/m/teamwork/t40511_asr/c/IEMOCAP_full_release/Session1/sentences/wav/Ses01F_script02_2/Ses01F_script02_2_F009.wav
File not found: /content/iemocap/session1/m/teamwork/t40511_asr/c/IEMOCAP_full_release/Session1/sentences/wav/Ses01F_script02_2/Ses01F_script02_2_F010.wav
File not found: /content/iemocap/session1/m/teamwork/t40511_asr/c/IEMO

Filter:   0%|          | 0/4446 [00:00<?, ? examples/s]

Filter:   0%|          | 0/1085 [00:00<?, ? examples/s]

Filter:   0%|          | 0/1085 [00:00<?, ? examples/s]

In [16]:
import os

def add_base_path(example):
    # Assuming the original paths are relative to /content/iemocap
    example['wav'] = os.path.join('/content/iemocap', example['wav'])
    return example

dataset = dataset.map(add_base_path, batched=False)

Map:   0%|          | 0/4446 [00:00<?, ? examples/s]

Map:   0%|          | 0/1085 [00:00<?, ? examples/s]

Map:   0%|          | 0/1085 [00:00<?, ? examples/s]

In [17]:
import librosa
def load_audio(example):
  audio_array, sample_rate = librosa.load(example['wav'], sr=16000)
  example["audio_array"] = audio_array
  example["sample_rate"] = sample_rate
  return example

In [18]:
dataset = dataset.map(load_audio, batched=False)
dataset

Map:   0%|          | 0/4446 [00:00<?, ? examples/s]

FileNotFoundError: [Errno 2] No such file or directory: '/content/iemocap/session1/m/teamwork/t40511_asr/c/IEMOCAP_full_release/Session2/sentences/wav/Ses02F_impro03/Ses02F_impro03_F000.wav'

In [20]:
dataset['train'][0].keys()

dict_keys(['id', 'wav', 'length', 'emo'])

In [24]:
import torch
from torch.nn.utils.rnn import pad_sequence
from sklearn.metrics import recall_score
import matplotlib.pyplot as plt
from sklearn.metrics import confusion_matrix
import seaborn as sns
import pandas as pd
import torch.nn.functional as F
import warnings
warnings.filterwarnings('ignore')


In [25]:
def update_iemocap_label(label, column="emo"):
    if label == "ang":
        return "anger"
    elif label == "hap":
        return "happiness"
    elif label == "neu":
        return "neutral"
    elif label == "sad":
        return "sadness"

In [26]:
def is_common(emotion):
    if emotion == "neutral" or emotion == "sadness" or emotion == "happiness" or emotion == "anger":
        return 1
    else:
        return 0

In [27]:
def update_iemocap_path(path):
    return "/m/triton/scratch/elec/t405-puhe/p/bijoym1/TempDatasets" + path.split("/m/teamwork/t40511_asr/c")[-1]


In [28]:

def to_label(emo, label2id):
    return label2id[emo]


In [40]:
def iemocap(session):
    train_json_file = f"/m/triton/scratch/elec/t405-puhe/p/bijoym1/iemocap/session{session}/train.json"
    test_json_file = f"/m/triton/scratch/elec/t405-puhe/p/bijoym1/iemocap/session{session}/test.json"

    with open(train_json_file, "r") as f:
        train_data = json.load(f)
    with open( test_json_file, "r") as f:
        test_data = json.load(f)

    train_df = pd.DataFrame.from_dict(train_data, orient="index").reset_index()
    train_df = train_df.rename(columns={'index': 'file_id', 'wav': 'audio'})

    test_df = pd.DataFrame.from_dict(test_data, orient="index").reset_index()
    test_df = test_df.rename(columns={'index': 'file_id', 'wav': 'audio'})

    train_df["audio"] = train_df["audio"].apply(update_iemocap_path).values
    test_df["audio"] = test_df["audio"].apply(update_iemocap_path).values

    train_df["emo"] = train_df["emo"].apply(update_iemocap_label)
    test_df["emo"] = test_df["emo"].apply(update_iemocap_label)

    labels = ['anger', 'happiness', 'neutral', 'sadness'] #["ang", "hap", "neu", "sad"]
    label2id, id2label = dict(), dict()

    for i, label in enumerate(labels):
        label2id[label] = str(i)
        id2label[str(i)] = label

    # convert categorical label to numerical labels
    train_df["label"] = train_df["emo"].apply(to_label, args=(label2id,))
    test_df["label"] = test_df["emo"].apply(to_label, args=(label2id,))

    train_audio_data = Dataset.from_pandas(train_df[['audio', 'label']])
    train_audio_data = train_audio_data.cast_column("audio", Audio(sampling_rate=16_000))

    test_audio_data = Dataset.from_pandas(test_df[['audio', 'label']])
    test_audio_data = test_audio_data.cast_column("audio", Audio(sampling_rate=16_000))

    ds = DatasetDict({
        'train' : train_audio_data,
        'test' : test_audio_data,
        'dev' : test_audio_data,
    })

    return label2id, id2label, ds

In [35]:
label2id, id2label, ds = iemocap(session=1)

In [36]:

# Now you can access label2id
print(label2id)

{'anger': '0', 'happiness': '1', 'neutral': '2', 'sadness': '3'}


In [37]:
train_audio_data = ds['train']

In [38]:
print(train_audio_data)

Dataset({
    features: ['audio', 'label'],
    num_rows: 4446
})


In [41]:
# Get the audio array and sampling rate for the first example
audio_array = train_audio_data[0]["audio"]["array"]
sampling_rate = train_audio_data[0]["audio"]["sampling_rate"]

print("Audio Array:", audio_array)
print("Sampling Rate:", sampling_rate)

FileNotFoundError: [Errno 2] No such file or directory: '/m/triton/scratch/elec/t405-puhe/p/bijoym1/TempDatasets/IEMOCAP_full_release/Session2/sentences/wav/Ses02F_impro03/Ses02F_impro03_F000.wav'